In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
import os
import time
from optimized_nonrigid_normcorre import OptimizedNonRigidCorrection, correct_motion

def main():
    """
    Example of how to use the NormCorre-style motion correction.
    """
    # Path to your data file - update this with your file path
    input_file_path = r"F:\Recovered\Research\BoninLab\PainModelingProject\calcium_imaging_data\CAAR Testing\CAAR part2 data\disinhibition\New folder (2)\Disinhib1_7.23.20_contra3_25um_cor.tif"
    
    # Set output directory
    output_dir = r"F:\Recovered\Research\BoninLab\PainModelingProject\calcium_imaging_data\CAAR Testing\Normcorreoutput"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Loading data from: {input_file_path}")
    try:
        # Load data
        video_data = imread(input_file_path)
        
        # Alternative loading options
        # For numpy arrays
        # video_data = np.load(input_file_path)
        
        # For HDF5 files
        # import h5py
        # with h5py.File(input_file_path, 'r') as f:
        #     video_data = f['dataset_name'][:]
    except Exception as e:
        print(f"Error loading data: {e}")
        return
    
    # Check if data loaded properly
    if video_data is None or video_data.size == 0:
        print("Failed to load data or data is empty")
        return
    
    print(f"Data loaded successfully: shape={video_data.shape}, dtype={video_data.dtype}")
    
    # Ensure data is 3D [frames, height, width]
    if len(video_data.shape) > 3:
        print(f"Data has {len(video_data.shape)} dimensions. Using first channel/volume.")
        # For 4D data [time, z, y, x] or [time, channel, y, x]
        if video_data.shape[1] <= 10:  # Likely channels or z-slices
            # Taking first channel/z-slice
            video_data = video_data[:, 0, :, :]
        else:
            print("Cannot determine how to reshape data. Please reshape data to [frames, height, width].")
            return
    
    # Convert to float32 if needed
    if video_data.dtype != np.float32:
        # Normalize to 0-1 range if data is integer type
        if np.issubdtype(video_data.dtype, np.integer):
            video_data = video_data.astype(np.float32) / np.iinfo(video_data.dtype).max
        else:
            video_data = video_data.astype(np.float32)
        print(f"Converted data to float32 with range [{video_data.min()}, {video_data.max()}]")
    
    # PERFORMANCE OPTIMIZATION SETTINGS
    # These settings can help if the algorithm is getting stuck
    
    # Reduce the number of patches by using larger patches and a smaller grid
    # This will make each patch computation faster while maintaining good coverage
    grid_size = (4, 4)  # Fewer patches = faster processing
    patch_size = (72, 72)  # Larger patches = more stable detection
    
    # Set maximum allowed correction to avoid extreme warping
    max_shifts = (15, 15)  # Limit shifts to 15 pixels
    
    # Process frames in smaller batches to avoid memory issues
    batch_size = 50  # Reduce from default 100
    
    # Adjust process count based on your hardware
    n_processes = max(1, os.cpu_count() // 2)  # Use half of available CPUs
    
    # Reduce iterations for faster processing
    iterations = 1  # Minimum iterations
    
    print("\nOPTIMIZED SETTINGS:")
    print(f"- Reduced grid size: {grid_size} (fewer patches)")
    print(f"- Increased patch size: {patch_size}")
    print(f"- Limited maximum shifts: {max_shifts} pixels")
    print(f"- Processing in batches of {batch_size} frames")
    print(f"- Using {n_processes} parallel processes")
    
    print("\nStarting optimized non-rigid motion correction...")
    start_time = time.time()
    
    corrected_video, shifts_dict, template = correct_motion(
        video_data,
        output_dir=output_dir,
        template=None,
        rigid_first=True,
        max_shifts=max_shifts,
        grid_size=grid_size,
        patch_size=patch_size,
        iterations=iterations,
        n_processes=n_processes,
        batch_size=batch_size,
        save_results=True,
        visualize=True
    )
    
    elapsed_time = time.time() - start_time
    print(f"\nMotion correction completed in {elapsed_time:.2f} seconds")
    print(f"Processing speed: {video_data.shape[0]/elapsed_time:.2f} frames/second")
    
    print(f"\nResults saved to {output_dir}")
    print(f"Corrected video saved as 16-bit TIFF: {os.path.join(output_dir, 'corrected_video.tif')}")
    print("Done!")

if __name__ == "__main__":
    main()

Loading data from: F:\Recovered\Research\BoninLab\PainModelingProject\calcium_imaging_data\CAAR Testing\CAAR part2 data\disinhibition\New folder (2)\Disinhib1_7.23.20_contra3_25um_cor.tif
Data loaded successfully: shape=(589, 1080, 1920), dtype=uint16
Converted data to float32 with range [0.0, 0.01879911497235298]

OPTIMIZED SETTINGS:
- Reduced grid size: (4, 4) (fewer patches)
- Increased patch size: (72, 72)
- Limited maximum shifts: (15, 15) pixels
- Processing in batches of 50 frames
- Using 4 parallel processes

Starting optimized non-rigid motion correction...
Starting optimized non-rigid motion correction...
Parameters:
 - Rigid first: True
 - Grid size: (4, 4)
 - Patch size: (72, 72)
 - Max shifts: (15, 15) pixels
 - Batch size: 50 frames
 - Parallel processes: 4
Creating template from 100 frames...
Applying rigid motion correction...
Rigid correction batch 1/12 (frames 0-49)...


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:20<00:00,  2.43it/s]


Rigid correction batch 2/12 (frames 50-99)...


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  4.09it/s]


Rigid correction batch 3/12 (frames 100-149)...


...

Rigid correction batch 11/12 (frames 500-549)...


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  3.91it/s]


Rigid correction batch 12/12 (frames 550-588)...


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:10<00:00,  3.67it/s]


Applying non-rigid motion correction...
Non-rigid correction batch 1/12 (frames 0-49, 16 patches per frame)...
Non-rigid correction batch 2/12 (frames 50-99, 16 patches per frame)...
Non-rigid correction batch 3/12 (frames 100-149, 16 patches per frame)...
Non-rigid correction batch 4/12 (frames 150-199, 16 patches per frame)...
Non-rigid correction batch 5/12 (frames 200-249, 16 patches per frame)...
Non-rigid correction batch 6/12 (frames 250-299, 16 patches per frame)...
Non-rigid correction batch 7/12 (frames 300-349, 16 patches per frame)...
Non-rigid correction batch 8/12 (frames 350-399, 16 patches per frame)...
Non-rigid correction batch 9/12 (frames 400-449, 16 patches per frame)...
Non-rigid correction batch 10/12 (frames 450-499, 16 patches per frame)...
Non-rigid correction batch 11/12 (frames 500-549, 16 patches per frame)...
Non-rigid correction batch 12/12 (frames 550-588, 16 patches per frame)...
Motion correction completed in 1717.79 seconds
Motion correction completed